In [68]:
import numpy as np
import math
import matplotlib.pyplot as plt
import imageio
from tqdm.notebook import tnrange, tqdm_notebook

## Set parameters

In [104]:
def set_u_kappa(u_p, kappa_p):
    global u, kappa, s, r
    u = u_p
    kappa = kappa_p
    s = u * dt / dx
    r = kappa * dt / (dx ** 2)
    print("u = {}, kappa = {}".format(u, kappa))
    print("r = {}, s = {}".format(r, s))
    

In [98]:
dx = 4e-7
dt = 0.01




teta = 5
a = 0
b = 1e-4

K = int((b - a) / dx) + 1

start = np.zeros(K, dtype=np.float64)
for i in range(K // 2):
    start[i] = 1.

## Define functions

In [99]:
def solve_linear(a, b, c, d):
    nf = len(d)
    ac, bc, cc, dc = map(np.array, (a, b, c, d))
    for it in range(1, nf):
        mc = ac[it - 1] / bc[it - 1]
        bc[it] = bc[it] - mc * cc[it - 1]
        dc[it] = dc[it] - mc * dc[it - 1]
    xc = bc
    xc[-1] = dc[-1] / bc[-1]

    for il in range(nf - 2, -1, -1):
        xc[il] = (dc[il] - cc[il] * xc[il + 1]) / bc[il]

    return xc

def get_dt(dx, u, kappa, coeff=0.9):
    return coeff * (dx ** 2) / (u * dx + 2 * kappa)




def left_border(t):
    return 0


def right_border(t):
    return 0


def solve(solver):
    t_iterations = int(teta / dt) + 1

    Ts = []
    Ts.append(start)

    for cur_t_iter in range(1, t_iterations + 1):
        t = dt * cur_t_iter
        cur = solver(t, Ts)
        Ts.append(cur)

    return Ts


def explicit_opposite(t, Ts):
    cur = np.zeros(K, dtype=np.float64)

    for i in range(K):
        prev_t = Ts[-1][i - 1] if i > 0 else left_border(t)
        next_t = Ts[-1][i + 1] if i < K - 1 else right_border(t)
        cur_t = Ts[-1][i]

        cur[i] = cur_t + r * (prev_t + next_t - 2 * cur_t)

    return cur


def explicit_nonopposite(t, Ts):
    cur = np.zeros(K, dtype=np.float64)

    for i in range(K):
        prev_t = Ts[-1][i - 1] if i > 0 else left_border(t)
        next_t = Ts[-1][i + 1] if i < K - 1 else right_border(t)
        cur_t = Ts[-1][i]

        cur[i] = cur_t - s * (next_t - cur_t) + r * (prev_t + next_t - 2 * cur_t)

    return cur


def confusion(t, Ts):
    #assert math.isclose(chi, 0.)

    if len(Ts) == 1:
        return explicit_opposite(t, Ts)

    cur = np.zeros(K, dtype=np.float64)

    for i in range(K):
        prev_t = Ts[-1][i - 1] if i > 0 else left_border(t)
        next_t = Ts[-1][i + 1] if i < K - 1 else right_border(t)
        cur_t = Ts[-2][i]

        cur[i] = cur_t - s * (next_t - prev_t)

    return cur


def implicit_opposite(t, Ts):
    a = np.zeros(K - 1, dtype=np.float64)
    b = np.zeros(K, dtype=np.float64)
    c = np.zeros(K - 1, dtype=np.float64)
    d = np.zeros(K, dtype=np.float64)

    for i in range(K):
        b[i] = 1 + s + 2 * r
        d[i] = Ts[-1][i]
    d[0] += left_border(t) * (r + s)
    d[K - 1] += right_border(t) * r
    for i in range(K - 1):
        a[i] = - r - s
        c[i] = -r
    return solve_linear(a, b, c, d)


def implicit_nonopposite(t, Ts):
    a = np.zeros(K - 1, dtype=np.float64)
    b = np.zeros(K, dtype=np.float64)
    c = np.zeros(K - 1, dtype=np.float64)
    d = np.zeros(K, dtype=np.float64)

    for i in range(K):
        b[i] = 1 - s + 2 * r
        d[i] = Ts[-1][i]
    d[0] += left_border(t) * r
    d[K - 1] -= right_border(t) * (s - r)

    for i in range(K - 1):
        a[i] = -r
        c[i] = s - r

    return solve_linear(a, b, c, d)


def plot_img(cur_index, Ts, xs, y_min, y_max):
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(xs, Ts[cur_index])

    ax.set_ylim(y_min, y_max)
    ax.set_xlim(0, 1e-4)
    

    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    plt.close(fig)

    return image



In [100]:
def run(method, filename, y_min='auto', y_max='auto'):
    Ts = solve(method)

    print(len(Ts))

    xs = [
        a + i * dx
        for i in range(K)
    ]
    
    if y_min == 'auto':
        y_min = max(-1e13, np.min(np.array(Ts)))
    
    if y_max == 'auto':
        y_max = min(1e13, np.max(np.array(Ts)))


    frames = [
        plot_img(i, Ts, xs, y_min * 0.9, y_max * 1.1)
        for i in tqdm_notebook(range(len(Ts)))
        if i % 4 == 0
    ]
    imageio.mimsave(filename, frames, fps=24)

In [101]:
set_u_kappa(9e-7, 0)
run(explicit_nonopposite, './kappa0/exp_nonopp.gif')

r = 0.0, s = 0.0225
502


## Kappa = 0, Explicit Non-opposite
![SegmentLocal](kappa0/exp_nonopp.gif "Explicit Non-opposite")

In [103]:
set_u_kappa(9e-5, 0)
run(explicit_opposite, './kappa0/exp_opp.gif')

r = 0.0, s = 2.2500000000000004
502


## Kappa = 0, Explicit Opposite
![SegmentLocal](kappa0/exp_opp.gif "Explicit Opposite")

In [105]:
set_u_kappa(9e-5, 0)
run(implicit_opposite, './kappa0/imp_opp.gif')

u = 9e-05, kappa = 0
r = 0.0, s = 2.2500000000000004
502


## Kappa = 0, Implicit Opposite
![SegmentLocal](kappa0/imp_opp.gif "Implicit Opposite")

In [108]:
set_u_kappa(0, 5e-12)
run(explicit_opposite, './u0/exp_opp_stable.gif')

u = 0, kappa = 5e-12
r = 0.3125, s = 0.0
502


## u = 0, r < 0.33 Explicit Opposite
![SegmentLocal](u0/exp_opp_stable.gif "Implicit Opposite")

In [ ]:
set_u_kappa(0, 9e-12)
run(explicit_opposite, './u0/exp_opp_unstable.gif')

## u = 0, r > 0.5 Explicit Opposite
![SegmentLocal](u0/exp_opp_unstable.gif "Implicit Opposite")

# Confusions etc

In [91]:
run(confusion, './kappa0/conf.gif')

502



502



502



502


In [94]:
run(confusion, './u0/conf.gif')

502



502



502



502



502


## u = 0, Confusion
![SegmentLocal](u0/conf.gif "Confusion")

## Kappa = 0, Confusion
![SegmentLocal](kappa0/conf.gif "Confusion")